# Little Lemon analysis and sales report


## Task 1:

* To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

* To find the information on the error, import the Error class from MySQL Connector/Python.

* Define your database configurations as a Python dictionary object called dbconfig.

* Establish a connection pool [pool_name = pool_b] with two connections. 

* Implement error handling using a try-except block in case the connection fails. 

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : ""
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_b
The pool size is: 2


**Task 2:**

Three guests are trying to book dinner slots simultaneously. Get the connections from `pool_b` and insert the following data in the `Bookings` table:

*Guest 1:*
- Table Number: 8
- First Name: Anees
- Last Name: Java
- Booking Time: 18:00
- EmployeeID: 6

*Guest 2:*
- Table Number: 5
- First Name: Bald
- Last Name: Vin
- Booking Time: 19:00
- EmployeeID: 6

*Guest 3:*
- Table Number: 12
- First Name: Jay
- Last Name: Kon
- Booking Time: 19:30
- EmployeeID: 6

Return all the connections back to the pool.

*Note: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a `PoolError`. Use try-except to print the error message.*

In [4]:
new_bookings_list = [
    {
    "TableNo" : 8,
    "GuestFirstName" : "Anees",
    "GuestLastName" : "Java",
    "BookingSlot" : "18:00:00",
    "EmployeeID" : 6
    },
    {
    "TableNo" : 5,
    "GuestFirstName" : "Bald",
    "GuestLastName" : "Vin",
    "BookingSlot" : "19:00:00",
    "EmployeeID" : 6
    },
    {
    "TableNo" : 12,
    "GuestFirstName" : "Jay",
    "GuestLastName" : "Kon",
    "BookingSlot" : "19:30:00",
    "EmployeeID" : 6
    }
]

# To add connection to the pool, the connection must be of MySQLConnection instance 
# Also possible to create via connect module and need the import below
import mysql.connector as connector

for i,new_booking in enumerate(new_bookings_list):
    insert_query = """
            INSERT INTO Bookings (TableNo, GuestFirstName, 
            GuestLastName, BookingSlot, EmployeeID)
            VALUES
            ({}, '{}', '{}', '{}', {});
        """.format(new_booking["TableNo"], new_booking["GuestFirstName"], new_booking["GuestLastName"],
                   new_booking["BookingSlot"], new_booking["EmployeeID"])
    
    try:
        guest_connected = pool.get_connection()
        print("[{} {}] is connected.\n".format(new_booking["GuestFirstName"], new_booking["GuestLastName"]))
        
        cursor = guest_connected.cursor()
        
        cursor.execute(insert_query)
        guest_connected.commit()

    except:
        print("No more connections are available.")
        print("Adding new connection in the pool.")
        
        # Create a connection
        connection=connector.connect(user="root",password="")
        # Add the connection into the pool
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
    
        guest_connected = pool.get_connection()
        print("[{} {}] is connected.\n".format(new_booking["GuestFirstName"], new_booking["GuestLastName"]))

        cursor = guest_connected.cursor()
        
        cursor.execute(insert_query)
        guest_connected.commit()

    # Let's close the cursor and the connection
    if guest_connected.is_connected():
        cursor.close()
        print("The cursor is closed.")
        guest_connected.close()
        print("MySQL connection is closed.")
    else:
        print("Connection is already closed")


No more connections are available.
Adding new connection in the pool.
A new connection is added in the pool.

[Anees Java] is connected.

The cursor is closed.
MySQL connection is closed.
[Bald Vin] is connected.

The cursor is closed.
MySQL connection is closed.
[Jay Kon] is connected.

The cursor is closed.
MySQL connection is closed.


**Task 3:**
- The name and EmployeeID of the Little Lemon manager.
- The name and role of the employee who receives the highest salary.
- The number of guests booked between 18:00 and 20:00.
- The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [5]:
#lets get connection

connection1 = pool.get_connection()
cursor = connection1.cursor()

In [6]:
# The name and EmployeeID of the Little Lemon manager.

select_query = """
SELECT Name,EmployeeID FROM Employees WHERE Role = 'Manager';
"""

cursor.execute(select_query)
column_names = [col_name for col_name in cursor.column_names]

print(column_names)

results = cursor.fetchall()

for result in results:
    print(result)


['Name', 'EmployeeID']
('Mario Gollini', 1)


In [8]:
#The name and role of the employee who receives the highest salary.

select_query = """
SELECT Name,Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1;
"""
cursor.execute(select_query)
column_names = [col_name for col_name in cursor.column_names]

print(column_names)

results = cursor.fetchone()

for result in results:
    print(result)

['Name', 'Role']
Mario Gollini
Manager


In [9]:
#The number of guests booked between 18:00 and 20:00.
select_query = """
SELECT COUNT(BookingID) AS nGuest FROM Bookings WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""
cursor.execute(select_query)
column_names = [col_name for col_name in cursor.column_names]

print(column_names)

results = cursor.fetchall()

for result in results:
    print(result)


['nGuest']
(7,)


In [10]:
#The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

select_query = """
SELECT CONCAT(B.GuestFirstName, " ", B.GuestLastName) AS GuestFullName, 
B.BookingID
FROM Bookings AS B
LEFT JOIN Employees as E
ON E.EmployeeID = B.EmployeeID
WHERE E.Role = "Receptionist"
ORDER BY B.BookingSlot DESC;
"""

print("The following guests are waiting to be seated:")

cursor.execute(select_query)
column_names = [col_name for col_name in cursor.column_names]

print(column_names)

results = cursor.fetchall()

for result in results:
    print(result)

The following guests are waiting to be seated:
['GuestFullName', 'BookingID']
('Jay Kon', 12)
('Bald Vin', 11)
('Anees Java', 10)


## Task 4:
Create a stored procedure named BasicSalesReport that returns the following statistics: 

- Total sales

- Average sale

- Minimum bill paid

- Maximum bill paid

In [11]:
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

stored_procedure_query = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount) AS Total_sales,
AVG(BillAmount) AS Average_sale, 
MIN(BillAmount) AS Minimum_bill_paid,
MAX(BillAmount) AS Maximum_bill_paid
FROM Orders;
END
"""

cursor.execute(stored_procedure_query)
print("Stored procedure named BasicSalesReport created.")

Stored procedure named BasicSalesReport created.


In [28]:
# Fetch the results and column names as you did
cursor.callproc("BasicSalesReport")
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names
columns = [column[0] for column in results.description]

# Format and print column names

for data in dataset:
    print('\t', columns[0], ':', data[0])
    print('\t', columns[1], ':', data[1])
    print('\t', columns[2], ':', data[2])
    print('\t', columns[3], ':', data[3])


	 Total_sales : 243
	 Average_sale : 48.6000
	 Minimum_bill_paid : 37
	 Maximum_bill_paid : 86


In [29]:
# Let's close the cursor and the connection
if connection1.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection1.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.


## Task 5:
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

- Get a connection from the pool.

- Create a buffered cursor.

- Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

- Returned the connection back to the pool.


The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [33]:
#lets get connection

connection1 = pool.get_connection()
cursor = connection1.cursor(buffered=True)


In [34]:
upcoming_bookings = """
SELECT B.BookingSlot,
CONCAT(B.GuestFirstName," ", B.GuestLastName) AS Guest_Name,
E.Name AS Emp_Name,
E.Role AS Emp_Role
FROM Bookings AS B
INNER JOIN Employees AS E
ON B.EmployeeID = E.EmployeeID
ORDER BY B.BookingSlot;
"""

cursor.execute(upcoming_bookings)
results = cursor.fetchmany(size=3)

for result in results:
    print("\nBookingSlot :", result[0])
    print("\tGuest_name :", result[1])
    print("\tAssigned to :", result[2],"[{}]".format(result[3]))


BookingSlot : 15:00:00
	Guest_name : Vanessa McCarthy
	Assigned to : Giorgos Dioudis [Head Chef]

BookingSlot : 17:30:00
	Guest_name : Marcos Romero
	Assigned to : Fatma Kaya [Assistant Chef]

BookingSlot : 18:00:00
	Guest_name : Anees Java
	Assigned to : John Millar [Receptionist]


In [35]:
# Let's close the cursor and the connection
if connection1.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection1.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
